In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sympy import (Matrix,
                   sin,
                   cos,
                   symbols,
                   MatrixSymbol,
                   diff,
                   simplify,
                   Eq,
                   linear_eq_to_matrix,
                   pprint)
from sympy.physics.mechanics import dynamicsymbols

In [2]:
# rho = np.array([0.5, 0.5, 0.5])
# m = np.array([250., 25., 25.])
# I = np.array([25., 2.5, 2.5])
# bs = b1 = b2 = 0. # parameters for non-conservative forces

C = lambda th: Matrix([[cos(th), -sin(th)], [sin(th), cos(th)]])
D = lambda th: Matrix([[sin(th), cos(th)], [-cos(th), sin(th)]])

In [3]:
t = symbols('t')

th_s, th_1, th_2, th_t = dynamicsymbols('theta_s theta_1 theta_2 theta_T')
dth_s, dth_1, dth_2, dth_t = dynamicsymbols('theta_s theta_1 theta_2, theta_T', 1)
ddth_s, ddth_1, ddth_2, ddth_t = dynamicsymbols('theta_s theta_1 theta_2, theta_T', 2)

q = Matrix([th_s, th_1, th_2, th_t])
qdot = Matrix([dth_s, dth_1, dth_2, dth_t])
qddot = Matrix([ddth_s, ddth_1, ddth_2, ddth_t])

y = q.row_insert(4, qdot)
ydot = qdot.row_insert(4, qddot)

tau_s = symbols('tau_s')
tau_1 = symbols('tau_1')
tau_2 = symbols('tau_2')

tau = Matrix([tau_s, tau_1, tau_2])

rho = symbols('rho_s rho_1 rho_2 rho_t')
m = symbols('m_s m_1 m_2 m_t')
I = symbols('I_s I_1 I_2 I_t')
b = symbols('b_s b_1 b_2')

# r_s = MatrixSymbol('r_s', 2, 1)
r_s0, r_s1 = symbols('r_s0 r_s1')
r_s = Matrix([r_s0, r_s1])

r_1 = r_s + C(y[0])@(Matrix([rho[0], 0.]) + C(y[1])@Matrix([rho[1], 0.]))
r_2 = r_1 + C(y[0])@C(y[1])@(Matrix([rho[1], 0.]) + C(y[2])@Matrix([rho[2], 0.]))
# r_t = MatrixSymbol('r_t', 2, 1)
r_t0, r_t1 = symbols('r_s0 r_s1')
r_t = Matrix([r_t0, r_t1])

r_c = r_t + C(y[3])@Matrix([0., rho[3]])

ee = r_1 + C(y[0])@C(y[1])@(Matrix([rho[1], 0.]) + C(y[2])@Matrix([2.*rho[2], 0.]))
J = ee.jacobian(q[:-1])

# v_s = Matrix(diff(r_s, t))
v_1 = Matrix(diff(r_1, t))
v_2 = Matrix(diff(r_2, t))
v_c = Matrix(diff(r_c, t))

T = 0.5*m[1]*v_1.dot(v_1) + 0.5*m[2]*v_2.dot(v_2) + \
    0.5*m[3]*v_c.dot(v_c) + 0.5*I[0]*y[4]**2 + 0.5*I[1]*y[5]**2 + \
    0.5*I[2]*y[6]**2 + 0.5*I[3]*y[7]**2
V = 0.

L = simplify(T - V)

In [4]:
dL_dth_s = diff(L, y[0])
dL_dth_1 = diff(L, y[1])
dL_dth_2 = diff(L, y[2])
dL_dth_t = diff(L, y[3])

dL_ddth_s = diff(L, y[4])
dL_ddth_1 = diff(L, y[5])
dL_ddth_2 = diff(L, y[6])
dL_ddth_t = diff(L, y[7])

dL_ddth_s_dt = diff(dL_ddth_s, t)
dL_ddth_1_dt = diff(dL_ddth_1, t)
dL_ddth_2_dt = diff(dL_ddth_2, t)
dL_ddth_t_dt = diff(dL_ddth_t, t)

th_s_eqn = Eq(simplify(dL_ddth_s_dt - dL_dth_s), tau[0] - b[0]*y[4])
th_1_eqn = Eq(simplify(dL_ddth_1_dt - dL_dth_1), tau[1] - b[1]*y[5])
th_2_eqn = Eq(simplify(dL_ddth_2_dt - dL_dth_2), tau[2] - b[2]*y[6])
th_t_eqn = Eq(simplify(dL_ddth_t_dt - dL_dth_t), 0.)

In [5]:
th_t_eqn

Eq(1.0*(I_t + m_t*rho_t**2)*Derivative(theta_T(t), (t, 2)), 0.0)

In [6]:
eqs = [th_s_eqn.lhs - th_s_eqn.rhs,
       th_1_eqn.lhs - th_1_eqn.rhs,
       th_2_eqn.lhs - th_2_eqn.rhs,
       th_t_eqn.lhs - th_t_eqn.rhs]

A, B = linear_eq_to_matrix(eqs, qddot[0], qddot[1], qddot[2], qddot[3])

In [7]:
from sympy import lambdify, matrix2numpy

dynamics = lambdify((y, tau, r_s, r_t, rho, m, I , b), matrix2numpy(A.LUsolve(B).T), "numpy")
pos_func = lambdify((y, r_s, r_t, rho), (matrix2numpy(r_1.T), matrix2numpy(r_2.T),  matrix2numpy(r_c.T)), "numpy")
vel_func = lambdify((y, r_s, r_t, rho), (matrix2numpy(v_1.T), matrix2numpy(v_2.T),  matrix2numpy(v_c.T)), "numpy")
ee_func = lambdify((y, r_s, rho), matrix2numpy(ee.T), "numpy")
J_func = lambdify((y, rho), matrix2numpy(J), "numpy")

In [8]:
y = np.zeros(8)
u = np.zeros(3)
u[1] = np.deg2rad(30.)

r_s = np.array([[0.], [0.]])
r_t = np.array([[2.], [0.]])

rho = np.array([0.5, 0.5, 0.5, 0.5])
m = np.array([250., 25., 25., 180.])
I = np.array([25., 2.5, 2.5, 18.])
b = np.zeros(3)

print(dynamics(y, u, r_s, r_t, rho, m, I, b))
print(pos_func(y, r_s, r_t, rho))
print(vel_func(y, r_s, r_t, rho))

[[-0.02223101  0.05148234 -0.04680213  0.        ]]
(array([[[3.],
        [0.]]]), array([[[4.],
        [0.]]]), array([[[2. ],
        [0.5]]]))
(array([[-0.,  0.]]), array([[0., 0.]]), array([[-0., -0.]]))


In [10]:
print(ee_func(y, r_s, rho))

[[[2.5]
  [0. ]]]


In [11]:
print(J_func(y, rho))

[[-0.  -0.  -0. ]
 [ 2.5  2.   1. ]]


In [12]:
import dill

with open('dynamics.pkl', "wb") as f:
    dill.dump(dynamics, f)

with open('position.pkl', "wb") as f:
    dill.dump(pos_func, f)

with open('velocity.pkl', "wb") as f:
    dill.dump(vel_func, f)

with open('ee.pkl', 'wb') as f:
    dill.dump(ee_func, f)

with open('Je.pkl', 'wb') as f:
    dill.dump(J_func, f)